Creating a visualization that goes frame by frame through an NFL play

In [25]:
# SELECT PLAY HERE
play = 4083
game = 2022100906

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
tracking_week_1 = pd.read_csv('../data/tracking_week_5.csv')
plays = pd.read_csv('../data/plays.csv')

In [23]:
df = pd.merge(tracking_week_1, plays, on=['gameId', 'playId'])

In [26]:
df = df[df['gameId'] == game]
df = df[df['playId'] == play]

In [27]:
df = df.sort_values(by='frameId')

In [28]:
df = df[['displayName', 'frameId', 'jerseyNumber', 'club', 'x', 'y', 'possessionTeam', 'defensiveTeam', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', ]]

In [29]:
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd

class NFLPlayVisualization:
    def __init__(self, dataframe):
        self.df = dataframe
        
        # Field dimensions
        self.FIELD_LENGTH = 120  # yards
        self.FIELD_WIDTH = 53.3  # yards
        
        # Setup the figure and axis
        plt.close('all')  # Close any existing plots
        self.fig, self.ax = plt.subplots(figsize=(12, 6))
        
        # Prepare the color mapping
        self.color_map = {
            'ARI':"red",
            'ATL':"red",
            'BAL':"purple",
            'BUF':"blue",
            'CAR':"blue",
            'CHI':"orange",
            'CIN':"orange",
            'CLE':"orange",
            'DAL':"blue",
            'DEN':"orange",
            'DET':"blue",
            'GB' :"green",
            'HOU':"red",
            'IND':"blue",
            'JAX':"brown",
            'KC' :"red",
            'LA' :"blue",
            'LAC':"yellow",
            'LV' :"black",
            'MIA':"teal",
            'MIN':"purple",
            'NE' :"silver",
            'NO' :"gold",
            'NYG':"blue",
            'NYJ':"green",
            'PHI':"green",
            'PIT':"yellow",
            'SEA':"blue",
            'SF' :"red",
            'TB' :"red",
            'TEN':"blue",
            'WAS':"red",
            'football':"brown"
        }
        
        # Setup the field
        self._setup_field()
        
    def _setup_field(self):
        """Setup the football field background"""
        self.ax.clear()
        self.ax.set_xlim(0, self.FIELD_LENGTH)
        self.ax.set_ylim(0, self.FIELD_WIDTH)
        
        # Field color
        self.ax.set_facecolor('green')
        
        # Field lines
        self.ax.axhline(0, color='white', linewidth=2)
        self.ax.axhline(self.FIELD_WIDTH, color='white', linewidth=2)
        
        # Yard lines
        for x in range(0, self.FIELD_LENGTH+1, 10):
            self.ax.axvline(x, color='white', linestyle='--', alpha=0.5)
        
        self.ax.set_title('NFL Play Visualization')
        self.ax.set_xlabel('Yards')
        self.ax.set_ylabel(' ')
        
    def animate(self, frame):
        """
        Animation function for matplotlib
        """
        # Clear and reset field
        self._setup_field()
        
        # Get data for current frame
        frame_data = self.df[self.df['frameId'] == self.unique_frames[frame]]
        
        # Plot players
        for _, player in frame_data.iterrows():
            color = self.color_map.get(player['club'], 'red')
            
            self.ax.scatter(
                player['x'], 
                player['y'], 
                color=color, 
                s=200,  # Size of the dot
                alpha=0.7,
                edgecolors='black'
            )
            
            # Add jersey number
            self.ax.text(
                player['x'], 
                player['y'], 
                str(int(player['jerseyNumber'])) if not pd.isna(player['jerseyNumber']) else '', 
                color='white', 
                ha='center', 
                va='center',
                fontweight='bold'
            )
        
        # Update title with current frame and game clock
        game_clock = frame_data['gameClock'].iloc[0] if not frame_data.empty else ''
        possessionTeam = frame_data['possessionTeam'].iloc[0] if not frame_data.empty else ''
        defensiveTeam = frame_data['defensiveTeam'].iloc[0] if not frame_data.empty else ''
        self.ax.set_title(f'NFL Play Visualization - Frame {frame+1} | Clock: {game_clock} | {possessionTeam} vs {defensiveTeam}')
        
        return self.ax
    
    def visualize(self, save_path='../gif/nfl_play.gif', interval=150):
        """
        Create the animation
        """
        # Get unique frames
        self.unique_frames = sorted(self.df['frameId'].unique())
        
        # Create animation
        anim = animation.FuncAnimation(
            self.fig, 
            self.animate, 
            frames=len(self.unique_frames),
            interval=interval,
            repeat=False
        )
        
        # Save the animation
        anim.save(save_path, writer='pillow', fps=10)
        
        # Close the plot to free up memory
        plt.close(self.fig)
        
        return anim

In [30]:
if not df.empty:
	viz = NFLPlayVisualization(df)
	viz.visualize()
else:
	print("The dataframe is empty. Please provide a dataframe with data.")